In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import json

from openai import OpenAI
from pydantic import BaseModel
from typing import Literal

## 1. Scrape Data from "Our Regional Cuisines" Website

In [2]:
# List of URLs to scrape
urls = [
    # Hokkaido
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/1083/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5962/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5971/index.html",

    # Aomori
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4892/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2507/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6875/index.html",

    # Iwate
    # "https://...",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/1130/index.html", 
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4368/index.html", #change picture URL for copyright reasons
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2978/index.html", #change picture URL for copyright reasons

    # Miyagi
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3760/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4643/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4667/index.html",

    # Akita
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2602/index.html", #change picture URL for copyright reasons
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/1215/index.html", #change picture URL for copyright reasons
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6976/index.html", #change picture URL for copyright reasons

    # Yamagata
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2748/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/937/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/540/index.html",

    # Fukushima
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5109/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4849/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6462/index.html",

    # Ibaraki
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3873/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/515/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2865/index.html",

    # Tochigi
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5818/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5824/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5822/index.html",

    # Gunma
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4066/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2936/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5285/index.html",

    # Saitama
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5849/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5026/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4527/index.html",

    # Chiba
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3072/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7185/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6147/index.html",

    # Tokyo
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7295/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4780/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6620/index.html",

    # Kanagawa
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4476/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4782/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4782/index.html",

    # Niigata
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6642/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7325/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6661/index.html",

    # Toyama
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7357/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2609/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6684/index.html",

    # Ishikawa
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7397/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6361/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2722/index.html",

    # Fukui
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4291/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2833/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6413/index.html",

    # Yamanashi
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6428/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2901/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/932/index.html",

    # Nagano
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4310/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2946/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2968/index.html",

    # Gifu
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4338/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6526/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7001/index.html",

    # Shizuoka
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7018/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/870/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7035/index.html",

    # Aichi
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6235/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5689/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5322/index.html",

    # Mie
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5716/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/821/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3195/index.html",

    # Shiga
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6689/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7179/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6708/index.html",

    # Kyoto
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5783/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6734/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4600/index.html",

    # Osaka
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6745/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4616/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7268/index.html",

    # Hyogo
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4866/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2680/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4620/index.html",

    # Nara
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7318/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7364/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4627/index.html",

    # Wakayama
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5304/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7391/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2701/index.html",

    # Tottori
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5577/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6242/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4913/index.html",

    # Shimane
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/666/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/384/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/2714/index.html",

    # Okayama
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7468/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4960/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3711/index.html",

    # Hiroshima
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7031/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5616/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/628/index.html",

    # Yamaguchi
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4447/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6281/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/614/index.html",

    # Tokushima
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7142/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6284/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5126/index.html",

    # Kagawa
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5074/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5426/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6296/index.html",

    # Ehime
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3974/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3283/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3977/index.html",

    # Kochi
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3336/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/1513/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6312/index.html",

    # Fukuoka
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6713/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5257/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3996/index.html",

    # Saga
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4315/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4006/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3346/index.html",

    # Nagasaki
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3351/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5413/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/5162/index.html",

    # Kumamoto
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/287/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6780/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4010/index.html",

    # Oita
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4017/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3363/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3362/index.html",

    # Miyazaki
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4019/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/3367/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6853/index.html",

    # Kagoshima
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/7492/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/4701/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6044/index.html",

    # Okinawa
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/173/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/168/index.html",
    "https://www.maff.go.jp/e/policies/market/k_ryouri/search_menu/6227/index.html",
]

In [3]:
def scrape_dish(url):
    image_base_url = "https://www.maff.go.jp/e/policies/market/k_ryouri/"
    webpage_source = requests.get(url).text
    content = BeautifulSoup(webpage_source, "html5lib")

    # Prefecture and dish name
    prefecture = content.select_one("h2.tit06.-prefecture span.pref span").get_text(strip=True)
    dish_name = content.select_one("h2.tit06 span.name").get_text(strip=True)

    # Image URL
    img_tag = content.select_one("div.menu_main img.resp_img")
    image_url = image_base_url + img_tag["src"].lstrip("../") if img_tag and img_tag.has_attr("src") else None

    # Area name (main lore area of the dish within the prefecture)
    area_el = content.find("h3", string=lambda s: s and "Main lore areas" in s)
    area_name = area_el.find_next("p").get_text(strip=True) if area_el else None

    # Main ingredients
    ingredients_el = content.find("h3", string=lambda s: s and "Main ingredients used" in s)
    main_ingredients = ingredients_el.find_next("p").get_text(strip=True) if ingredients_el else None

    # History and culture text
    history_el = content.find("h3", string=lambda s: s and "History/origin/related events" in s)
    history_text = history_el.find_next("p").get_text(" ", strip=True) if history_el else None

    # Recipe (ingredient -> amount)
    recipe = {}
    for li in content.select("div.material-list li"):
        name = li.select_one("span.name")
        qty = li.select_one("span.quantity")
        if name and qty:
            ingredient = name.get_text(strip=True)
            # Replace the Japanese full-width slash with a normal one
            amount = qty.get_text(strip=True).replace("ï¼", "/")
            recipe[ingredient] = amount

    # Number of servings (first number)
    servings = 1 # defaults to one (if no number can be extracted)
    serving_el = content.select_one("div.material__wrap h4 span")
    if serving_el:
        serving_text = serving_el.get_text(strip=True)
        for char in serving_text:
            if char.isdigit():
                servings = int(char)
                break

    # Return as dictionary
    return {
        "dish_name": dish_name,
        "prefecture": prefecture,
        "area_name": area_name,
        "main_ingredients": main_ingredients,
        "history": history_text,
        "recipe": recipe,
        "servings": servings,
        "image_url": image_url
    }

In [4]:
# Scrape all URLs
all_dishes = []

for url in urls:
    dish = scrape_dish(url)
    all_dishes.append(dish)
    time.sleep(5)  # wait 5 seconds between requests

In [5]:
#Save scraped data to a JSON file
with open("all_dishes_scraped.json", "w", encoding="utf-8") as f:
    json.dump(all_dishes, f, ensure_ascii=False, indent=4)

## 2. Enrich data with GPT (classify dishes, calculate calories, summarize long texts)

In [6]:
#read JSON back into jupyter notebook
with open("all_dishes_scraped.json", "r", encoding="utf-8") as f:
    all_dishes = json.load(f)

In [7]:
# Set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = "..."

In [8]:
client = OpenAI() #create the client

class DishInfo(BaseModel):
    type: Literal[
        "rice dish",
        "noodle dish",
        "soup",
        "meat dish",
        "fish dish",
        "pickles",
        "vegetable",
        "sweets",
        "other"
    ]
    seasonality: Literal[
        "all season",
        "spring",
        "summer",
        "fall",
        "winter"
    ]
    calories: int           # in kcal
    protein: int            # in grams
    carbohydrates: int      # in grams
    fats: int               # in grams
    sodium: int             # in grams
    vegan: bool
    vegetarian: bool
    no_gluten: bool
    no_seafood: bool
    no_pork: bool
    no_dairy: bool
    no_nuts: bool

In [9]:
#classifies dishes and calculates calories based on amounts of ingredients
def enrich_dish(dish):
    # Convert recipe dict to a formatted string
    ingredients_list = [f"{name} {amount}" for name, amount in dish["recipe"].items()]
    ingredients_text = ". ".join(ingredients_list)

    prompt_content = (
        f"Dish name: {dish['dish_name']}. Ingredients: {ingredients_text}. "
    )

    #make a request
    response = client.responses.parse(
        model="gpt-5-nano",
        input=[
            {"role": "system", "content": (
                "Please classify the type and seasonality of this dish, estimate its total nutritional values, and determine dietary restrictions. "
                "Return calories (kcal), protein (g), carbohydrates (g), fats (g), and sodium (g)."
            )},
            {"role": "user", "content": prompt_content},
        ],
        text_format=DishInfo,
    )

    return response.output_parsed

In [10]:
#summarizes the history text of a dish to a maximum of 150 words
def summarize_history(dish):
    prompt_content = (
        f"Please summarize the following text about the dish '{dish['dish_name']}' "
        f"to be maximum 150 words, keeping the most important information and cultural background.\n\n"
        f"'{dish['history']}'"
    )

    response = client.responses.create(
        model="gpt-5-nano",
        input=[
            {"role": "system", "content": "You are an expert in summarizing text about dishes."},
            {"role": "user", "content": prompt_content},
        ]
    )

    dish["history"] = response.output_text  # replace the existing history

In [11]:
# Loop through all dishes
for dish in all_dishes:
    
    dish_info = enrich_dish(dish)
    
    # Update the original dish dict
    dish.update(dish_info.dict())
    
    # summarize history if longer than 150 words
    if len(dish['history'].split()) > 150:
        summarize_history(dish)

In [12]:
#Save enriched data to a JSON file
with open("all_dishes_enriched.json", "w", encoding="utf-8") as f:
    json.dump(all_dishes, f, ensure_ascii=False, indent=4)

## 3. Query Google Places API for dish origin coordinates and places where a dish can be eaten

In [13]:
#read JSON back into jupyter notebook
with open("all_dishes_enriched.json", "r", encoding="utf-8") as f:
    all_dishes = json.load(f)

In [14]:
PLACES_API_KEY = "..."

In [15]:
def query_coordinates(area_name, prefecture):
    url = "https://places.googleapis.com/v1/places:searchText"

    # Try querying the corrdinates of the specific area
    text_query_area = f"{area_name}, {prefecture}, Japan"
    payload = {"textQuery": text_query_area}
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": PLACES_API_KEY,
        "X-Goog-FieldMask": (
            "places.displayName,"
            "places.location.latitude,"
            "places.location.longitude"
        )
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()
        places = data.get("places", [])

        if places:
            first_place = places[0]
            lat = first_place["location"]["latitude"]
            lon = first_place["location"]["longitude"]
            return lat, lon

        # Fallback to prefecture only if no coordinates for specific area were found
        text_query_pref = f"{prefecture}, Japan"
        payload = {"textQuery": text_query_pref}

        response = requests.post(url, headers=headers, json=payload)
        data = response.json()
        places = data.get("places", [])

        if places:
            first_place = places[0]
            lat = first_place["location"]["latitude"]
            lon = first_place["location"]["longitude"]
            return lat, lon

        #if neither specific area nor prefecture could be found
        return None, None

    except Exception as e:
        print(f"Error querying coordinates for {area_name}: {e}")
        return None, None

In [16]:
# Loop through all dishes and add coordinates
for dish in all_dishes:
    area_name = dish.get("area_name")
    prefecture = dish.get("prefecture")

    lat, lon = query_coordinates(area_name, prefecture)

    # Update the dish with coordinates
    dish["area_lat"] = lat
    dish["area_lon"] = lon


In [18]:
def query_places(dish_name, area_name, prefecture, max_results=5):
    def is_in_japan(lat, lon):
        # Rough bounding box for Japan
        return 24.0 <= lat <= 46.0 and 122.0 <= lon <= 154.0

    text_query = f"{dish_name} {area_name} {prefecture} Japan"
    url = "https://places.googleapis.com/v1/places:searchText"

    payload = {"textQuery": text_query}

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": PLACES_API_KEY,
        "X-Goog-FieldMask": (
            "places.id,"
            "places.displayName,"
            "places.location.latitude,"
            "places.location.longitude,"
            "places.rating,"
            "places.priceLevel,"
            "places.googleMapsUri"
        ),
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()
        places = data.get("places", [])

        if not places:
            return []

        top_places = [
            {
                "id": p.get("id"),
                "name": p.get("displayName", {}).get("text", ""),
                "latitude": p.get("location", {}).get("latitude"),
                "longitude": p.get("location", {}).get("longitude"),
                "rating": p.get("rating"),
                "price_level": p.get("priceLevel"),
                "googleMapsUri": p.get("googleMapsUri"),
            }
            for p in places
        ]

        # Keep only places located within Japan
        filtered_places = [
            p
            for p in top_places
            if p["latitude"] is not None
            and p["longitude"] is not None
            and is_in_japan(p["latitude"], p["longitude"])
        ]

        return filtered_places[:max_results]

    except Exception as e:
        print(f"Error querying places for {dish_name}: {e}")
        return []


In [19]:
all_places = []  # list of dicts

#loop through all dishes and add places where dish can be eaten
for dish in all_dishes:
    places = query_places(dish["dish_name"], dish["area_name"], dish["prefecture"])
    dish["places"] = [p["id"] for p in places] if places else []
    all_places.extend(places)
    time.sleep(1)  # polite delay

In [20]:
#Save data to a JSON file
with open("all_dishes_with_places.json", "w", encoding="utf-8") as f:
    json.dump(all_dishes, f, ensure_ascii=False, indent=4)

In [21]:
#Save data to a JSON file
with open("all_places.json", "w", encoding="utf-8") as f:
    json.dump(all_places, f, ensure_ascii=False, indent=4)

## 4. Final processing and saving data as CSV-File

In [3]:
#read JSON back into jupyter notebook
with open("all_dishes_with_places.json", "r", encoding="utf-8") as f:
    all_dishes = json.load(f)

In [4]:
#read JSON back into jupyter notebook
with open("all_places.json", "r", encoding="utf-8") as f:
    all_places = json.load(f)

In [5]:
# Convert lists of dictionaries to DataFrames
df_dishes = pd.DataFrame(all_dishes)
df_places = pd.DataFrame(all_places)

In [7]:
# Compute nutrients per serving
for nutrient in ["calories", "protein", "carbohydrates", "fats", "sodium"]:
    df_dishes[nutrient] = df_dishes[nutrient] / df_dishes["servings"]

# Drop columns not needed
df_dishes = df_dishes.drop(columns=["servings", "recipe",])

In [32]:
# change images to adhere to copyright rules
df_dishes.loc[df_dishes["dish_name"] == "Samma no Surimi Jiru (Pacific Saury Fish Ball Soup)", "image_url"] = "https://www.maff.go.jp/e/policies/market/k_ryouri/assets/uploads/2023/11/iwate_18_2.jpeg"
df_dishes.loc[df_dishes["dish_name"] == "Mame-shitogi", "image_url"] = "https://www.maff.go.jp/e/policies/market/k_ryouri/assets/uploads/2023/08/iwate_8_2.jpg"
df_dishes.loc[df_dishes["dish_name"] == "Kiritanpo Nabe", "image_url"] = "https://www.maff.go.jp/e/policies/market/k_ryouri/assets/uploads/2023/08/akita_6_4.jpg"
df_dishes.loc[df_dishes["dish_name"] == "Damako nabe", "image_url"] = "https://www.maff.go.jp/e/policies/market/k_ryouri/assets/uploads/2022/08/akita_4_3.jpg"
df_dishes.loc[df_dishes["dish_name"] == "Aka-zuke/Aka-zushi (Red Shiso sushi)", "image_url"] = "https://www.maff.go.jp/e/policies/market/k_ryouri/assets/uploads/2024/02/akita_29_2.jpg"

In [34]:
df_dishes.head()

,dish_name,prefecture,area_name,main_ingredients,history,image_url,type,seasonality,calories,protein,...,vegan,vegetarian,no_gluten,no_seafood,no_pork,no_dairy,no_nuts,area_lat,area_lon,places
0,Kobumaki,Hokkaido,Hidaka region,"Kelp, salmon, herring, kanpyo (dried gourd)",Kombu-maki (kelp rolls) originated in Hokkaido...,https://www.maff.go.jp/e/policies/market/k_ryo...,fish dish,all season,450.0,9.50,...,False,False,False,False,True,True,True,42.438173,142.636768,[]
1,Sanpei-Jiru,Hokkaido,All of Hokkaido Prefecture,"Salmon, Cod, Herring, Potatoes, Daikon Radish",Sanpei-jiru is a traditional Hokkaido stew of ...,https://www.maff.go.jp/e/policies/market/k_ryo...,soup,all season,246.5,10.50,...,False,False,True,False,True,True,True,43.220327,142.863474,[ChIJ__8PSIMpC18RwV3l2mTGVgE]
2,Jingisukan (Lamb and Vegetable Hot Pot),Hokkaido,All of Hokkaido Prefecture,"Lamb, Onion, Bean sprouts, Green bell pepper, ...","Jingisukan, a lamb and vegetable hot pot iconi...",https://www.maff.go.jp/e/policies/market/k_ryo...,meat dish,all season,440.0,36.25,...,False,False,False,True,True,True,True,43.220327,142.863474,[]
3,Hoshi Mochi(Dried rice cake),Aomori,Tsugaru Region,"sticky rice, Sugar",It has long been a preserved food made during ...,https://www.maff.go.jp/e/policies/market/k_ryo...,sweets,all season,4460.5,79.50,...,True,True,True,True,True,True,True,40.822220,140.747352,[]
4,Kinka mochi,Aomori,southwestern region,"Flour, walnuts, miso, brown sugar.",Kinka mochi is a traditional southern Japanese...,https://www.maff.go.jp/e/policies/market/k_ryo...,sweets,all season,1840.0,43.00,...,True,True,False,True,True,True,False,40.765708,140.917588,"[ChIJQcC39R-fm18R6j9xV-cGqBM, ChIJ3bJcqB6fm18R..."


In [35]:
df_places.head()

,id,name,latitude,longitude,rating,price_level,googleMapsUri
0,ChIJ__8PSIMpC18RwV3l2mTGVgE,Aji no Sanpei,43.058815,141.353627,4.0,PRICE_LEVEL_MODERATE,https://maps.google.com/?cid=96482578506538433...
1,ChIJQcC39R-fm18R6j9xV-cGqBM,A-FACTORY,40.829997,140.734927,4.3,PRICE_LEVEL_MODERATE,https://maps.google.com/?cid=14163896734822727...
2,ChIJ3bJcqB6fm18RruCZ6EBTv-Q,Aomori Gyosai Center (Aomori Nokkedon),40.825897,140.735912,4.1,None,https://maps.google.com/?cid=16482984699444715...
3,ChIJrbb982eum18RXrO4USJsKv0,Tsugarufujimiko reservoir,40.750863,140.386633,4.2,None,https://maps.google.com/?cid=18242512135273362...
4,ChIJ3wDtCfKnm18RDp1XaSSvpys,Roadside Station Namioka Apple Hill,40.717661,140.579487,3.8,None,https://maps.google.com/?cid=31456754356626137...


In [36]:
# Save each DataFrame to CSV
df_dishes.to_csv("all_dishes.csv", index=False, encoding="utf-8-sig")
df_places.to_csv("all_places.csv", index=False, encoding="utf-8-sig")